In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_palette("GnBu_d")
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime
from datetime import timedelta
import math
import random 
import timeit
from haversine import haversine # conda install -c conda-forge haversine ''

In [2]:
BRM = pd.read_csv("resources/bremen.csv")
FRB = pd.read_csv("resources/freiburg.csv")

In [3]:
BRM["trip_duration"] = BRM["trip_duration"].astype(str).str.extract("days (.*?)\.")
FRB["trip_duration"] = FRB["trip_duration"].astype(str).str.extract("days (.*?)\.")

Ab hier:

To work with the duration of the Trips we need to convert them to a useful type:

In [4]:
BRM["trip_duration"] = BRM["trip_duration"].apply(pd.Timedelta)
FRB["trip_duration"] = FRB["trip_duration"].apply(pd.Timedelta)

After the convertion we can sum the time daily driven:

In [5]:
BRM_daily_driven = BRM.groupby("day").agg({"trip_duration":"sum"})
FRB_daily_driven = FRB.groupby("day").agg({"trip_duration":"sum"})

For a more detailed aggregation we add a daytime to the variables, so we can calculate time driven by daytime. This also allows a better mapping of how weather effects demand.

In [6]:
BRM["daytime"] = "null"
FRB["daytime"] = "null"

Define Daytimes: Morning: 6 - 11 | Noon: 11 - 16 | Evening: 16 - 21 | Night: 21 - 6

In [17]:
BRM_morning = BRM[(BRM["time"] > "06:00:00") & (BRM["time"] <= "11:00:00")] #saving rows where time is in defined range to a new df
BRM_noon = BRM[(BRM["time"] > "11:00:00") & (BRM["time"] <= "16:00:00")]
BRM_evening = BRM[(BRM["time"] > "16:00:00") & (BRM["time"] <= "21:00:00")]
BRM_night = BRM[(BRM["time"] > "21:00:00")] + BRM[(BRM["time"] <= "06:00:00")]

In [8]:
FRB_morning = FRB[(FRB["time"] > "06:00:00") & (FRB["time"] <= "11:00:00")]
FRB_noon = FRB[(FRB["time"] > "11:00:00") & (FRB["time"] <= "16:00:00")]
FRB_evening = FRB[(FRB["time"] > "16:00:00") & (FRB["time"] <= "21:00:00")]
FRB_night = FRB[(FRB["time"] > "21:00:00")] + FRB[(FRB["time"] <= "06:00:00")]

adding corresponding values to column:

In [14]:
for i in BRM_morning.index: # using indexes from lists above to cycle through the whole data
   BRM.at[i , "daytime"] = 0 # morning refers to 0
for i in BRM_noon.index:
   BRM.at[i , "daytime"] = 1 # noon -> 1
for i in BRM_evening.index:
   BRM.at[i , "daytime"] = 2 # evening -> 2
for i in BRM_night.index:
   BRM.at[i , "daytime"] = 3 # night -> 3

In [15]:
for i in FRB_morning.index:
   FRB.at[i , "daytime"] = 0
for i in FRB_noon.index:
   FRB.at[i , "daytime"] = 1
for i in FRB_evening.index:
   FRB.at[i , "daytime"] = 2
for i in FRB_night.index:
   FRB.at[i , "daytime"] = 3

Summarizing data over day and daytime via aggregation of the trip durations taken.

In [11]:
BRM_daytime_driven = BRM.groupby(["day","daytime"]).agg({"trip_duration":"sum"})
BRM_daytime_driven["trip_duration"] = BRM_daytime_driven["trip_duration"].astype(str).str.extract("days (.*?)\.")

In [12]:
FRB_daytime_driven = FRB.groupby(["day","daytime"]).agg({"trip_duration":"sum"})
FRB_daytime_driven["trip_duration"] = FRB_daytime_driven["trip_duration"].astype(str).str.extract("days (.*?)\.")

In [13]:
# Anzeige von dem ganzen gedöns das ich ausgerechnet hab und ein Auschnitt von einem Tag aus Freiburg zur Überprüfung:
print(BRM_daytime_driven)
print(FRB_daytime_driven)
print(FRB[FRB["day"] == "2019-05-07"])

                   trip_duration
day        daytime              
2019-01-20 0            04:21:00
           1            20:37:00
           2            08:41:00
           3            05:38:00
2019-01-21 0            16:45:00
...                          ...
2020-01-19 3            19:32:00
2020-01-20 0            21:32:00
           1            21:54:00
           2            17:04:00
           3            06:35:00

[1451 rows x 1 columns]
                   trip_duration
day        daytime              
2019-05-06 1            00:07:00
2019-05-07 0            00:07:00
           1            00:32:00
2019-05-08 0            00:03:00
           1            00:18:00
...                          ...
2020-01-19 3            02:26:00
2020-01-20 0            05:46:00
           1            22:31:00
           2            14:34:00
           3            12:07:00

[1008 rows x 1 columns]
          day      time  b_number      city trip_duration   orig_lat  \
1  2019-05-07  10:42